In [2]:
import torch
import h5py
import os
import numpy as np
import matplotlib.pyplot as plt


find all files

In [4]:
def get_all_hdf5_files(directory):
    # 获取目录下所有以 .h5 结尾的文件路径
    file_paths = []
    for root, dirs, files in os.walk(directory):
        for file in files:
            if file.endswith(f'.hdf5'):  # 只获取 .h5 文件
                file_paths.append(os.path.join(root, file))
    return file_paths

# 示例
directory = './grab_both_cubes_v1'  # 替换为你的数据文件夹路径
file_paths = get_all_hdf5_files(directory)

save_directory = './grab_both_cubes_v2'
print(len(file_paths))
print(file_paths)

51
['./grab_both_cubes_v1\\episode_0.hdf5', './grab_both_cubes_v1\\episode_1.hdf5', './grab_both_cubes_v1\\episode_10.hdf5', './grab_both_cubes_v1\\episode_11.hdf5', './grab_both_cubes_v1\\episode_12.hdf5', './grab_both_cubes_v1\\episode_13.hdf5', './grab_both_cubes_v1\\episode_14.hdf5', './grab_both_cubes_v1\\episode_17.hdf5', './grab_both_cubes_v1\\episode_19.hdf5', './grab_both_cubes_v1\\episode_2.hdf5', './grab_both_cubes_v1\\episode_20.hdf5', './grab_both_cubes_v1\\episode_22.hdf5', './grab_both_cubes_v1\\episode_23.hdf5', './grab_both_cubes_v1\\episode_24.hdf5', './grab_both_cubes_v1\\episode_25.hdf5', './grab_both_cubes_v1\\episode_26.hdf5', './grab_both_cubes_v1\\episode_27.hdf5', './grab_both_cubes_v1\\episode_28.hdf5', './grab_both_cubes_v1\\episode_29.hdf5', './grab_both_cubes_v1\\episode_3.hdf5', './grab_both_cubes_v1\\episode_30.hdf5', './grab_both_cubes_v1\\episode_31.hdf5', './grab_both_cubes_v1\\episode_32.hdf5', './grab_both_cubes_v1\\episode_33.hdf5', './grab_both_cub

get max num of the data

In [5]:
max_nums = 0
for file_path in file_paths:
    with h5py.File(file_path, 'r') as f:
        action = f['action'][:]
        # print(action.shape[0])
        if action.shape[0] > max_nums:
            max_nums = action.shape[0]
            
print(max_nums)

545


get all keys

In [6]:
obj_names = []

def print_hdf5_structure(obj, indent=0):
    """递归打印HDF5文件中的结构"""
    if isinstance(obj, h5py.Dataset):
        # 如果是数据集，打印数据集信息
        print('  ' * indent + f"Dataset: {obj.name}")
        obj_names.append(obj.name)
        print('  ' * indent + f"  Shape: {obj.shape}")
        # print('  ' * indent + f"  size: {obj.shape}")
        # print('  ' * indent + f"  Data type: {obj.dtype}")
    elif isinstance(obj, h5py.Group):
        # 如果是组，打印组信息并递归其内容
        print('  ' * indent + f"Group: {obj.name}")
        for key in obj.keys():
            print_hdf5_structure(obj[key], indent + 1)
            
# no appending obj_names
def print_hdf5_structure_pure(obj, indent=0):
    """递归打印HDF5文件中的结构"""
    if isinstance(obj, h5py.Dataset):
        # 如果是数据集，打印数据集信息
        print('  ' * indent + f"Dataset: {obj.name}")
        print('  ' * indent + f"  Shape: {obj.shape}")
        # print('  ' * indent + f"  size: {obj.shape}")
        # print('  ' * indent + f"  Data type: {obj.dtype}")
    elif isinstance(obj, h5py.Group):
        # 如果是组，打印组信息并递归其内容
        print('  ' * indent + f"Group: {obj.name}")
        for key in obj.keys():
            print_hdf5_structure_pure(obj[key], indent + 1)
            
            
with h5py.File(file_paths[0],'r') as f:
    print_hdf5_structure(f)
    
print(obj_names)

Group: /
  Dataset: /action
    Shape: (418, 7)
  Group: /cam_data
    Dataset: /cam_data/human_camera
      Shape: (418, 240, 426, 3)
    Dataset: /cam_data/robot_camera
      Shape: (418, 240, 426, 3)
  Dataset: /end_position
    Shape: (418, 6)
  Dataset: /gripper_state
    Shape: (418,)
  Dataset: /qpos
    Shape: (418, 7)
  Dataset: /qvel
    Shape: (418, 7)
  Dataset: /step
    Shape: (418,)
  Dataset: /timestamp
    Shape: (418,)
  Dataset: /transformed_hand_coords
    Shape: (418, 24, 3)
  Dataset: /transformed_hand_frames
    Shape: (418, 4, 3)
['/action', '/cam_data/human_camera', '/cam_data/robot_camera', '/end_position', '/gripper_state', '/qpos', '/qvel', '/step', '/timestamp', '/transformed_hand_coords', '/transformed_hand_frames']


get data and transfer them

In [80]:
# print some examples 

# with h5py.File(r'./grab_cube2_v1/episode_0.hdf5', 'r') as f:
#     for i in [0,99,170]:
#         for obj_name in obj_names:
#             if obj_name == "/cam_data/human_camera" :
#                 img = f[obj_name][i]
                
#                 # 使用 matplotlib 显示图像
#                 plt.imshow(img)
#                 plt.title(f"Human Camera - Image {i}")
#                 plt.axis('off')  # 不显示坐标轴
#                 plt.show()
                    
                
                
#             elif obj_name == "/cam_data/robot_camera" :
#                 img = f[obj_name][i]
                
#                 # 使用 matplotlib 显示图像
#                 plt.imshow(img)
#                 plt.title(f"Robot Camera - Image {i}")
#                 plt.axis('off')  # 不显示坐标轴
#                 plt.show()
            
#             else:
#                 print(f"{i}th {obj_name}: {f[obj_name][i]}")
#         print("\n")
    

In [8]:
idx = 0
# 压缩级别，1到9之间，9为最大压缩比
compression_level = 6

for file_path in file_paths:
    
    with h5py.File(file_path, 'r') as f, \
    h5py.File(save_directory + f"/episode_{idx}.hdf5",'w') as new_f:
    # with h5py.File('./grab_cube2_v1/episode_0.hdf5', 'r') as f, \
    # h5py.File('./grab_cube2_v2/episode_test.hdf5','w') as new_f:
        print(f"saving {file_path}")
        idx += 1
        for name in obj_names:
            if name == '/qpos':
                data = f['/end_position'][:]
                print(data.shape)
                gripper_state = f['/gripper_state'][:]
                
                
                gripper_state_tensor = torch.tensor(gripper_state, dtype=torch.float32)
                data_tensor = torch.tensor(data, dtype=torch.float32)
                
                if gripper_state_tensor.dim() == 1:
                    gripper_state_tensor = gripper_state_tensor.unsqueeze(1)  # 转为 [num_samples, 1]
                
                data = torch.cat((data_tensor, gripper_state_tensor), dim=1)
                
                print(data.shape)
                
                # print(data)
            
            elif name == '/action':
                
                
                action = f[name][:]
                # print(action.shape)
                gripper_state = f['/gripper_state'][:]
                
                gripper_state_tensor = torch.tensor(gripper_state, dtype=torch.float32)
                action_tensor = torch.tensor(action, dtype=torch.float32)
                # # 创建一个形状为 (180, 1) 的全零张量
                # zeros = torch.zeros(action_tensor.size(0), 1)

                # # 在第二维拼接
                # action_tensor = torch.cat((action_tensor, zeros), dim=1)
            
                                
                # 确保 gripper_state_tensor 是 2D 张量
                if gripper_state_tensor.dim() == 1:
                    gripper_state_tensor = gripper_state_tensor.unsqueeze(1)  # 转为 [num_samples, 1]
                
                
                # print(action_tensor.shape)
                # print(gripper_state.shape)
                
                # 拼接 states_tensor 和 gripper_state_tensor
                data = torch.cat((action_tensor, gripper_state_tensor), dim=1)
                
            
                

            else:
                data = f[name][:]
            
            # print(f'{name}: {data.shape}')
            # 确保 std_shape 是可以修改的，并调整第一个维度
            # std_shape = list(data.shape)  # 将元组转换为列表以便修改
            # std_shape[0] = max_nums  # 修改第一个维度为 max_nums
            # std_shape = tuple(std_shape)  # 再转换回元组
            num = data.shape[0]
            dim = data.shape[1:]
            if num < max_nums:
                # padded_data = np.zeros(dim)
                # 将原始数据复制到填充后的数组中
                # padded_data[:num] = data
                listed_data = list(data)
                # 存储最后一个数据点
                padded_data = listed_data[num-1]

                # 使用最后一个数据填充
                for i in range(num, max_nums):
                    listed_data.append(padded_data)
                padded_data = tuple(listed_data)
                # 使用最后一个有效数据点进行填充
                # padded_data[num:] = data[-1]  # 填充剩余部分为最后一个有效数据点
            else:
                padded_data = data
            
            # 步骤 4: 将填充后的数据写入新文件
            # new_f.create_dataset(name, data=padded_data)
            # new_f.create_dataset(name, data=data, compression='gzip', compression_opts=compression_level)
            new_f.create_dataset(name, data=padded_data, compression='gzip', compression_opts=compression_level)
            # print(f"{name}: {len(padded_data)} successfully saved")
        print(f"{file_path} successfully saved in {save_directory}/episode_{idx - 1}.hdf5")

saving ./grab_both_cubes_v1\episode_0.hdf5
(418, 6)
torch.Size([418, 7])
./grab_both_cubes_v1\episode_0.hdf5 successfully saved in ./grab_both_cubes_v2/episode_0.hdf5
saving ./grab_both_cubes_v1\episode_1.hdf5
(538, 6)
torch.Size([538, 7])
./grab_both_cubes_v1\episode_1.hdf5 successfully saved in ./grab_both_cubes_v2/episode_1.hdf5
saving ./grab_both_cubes_v1\episode_10.hdf5
(420, 6)
torch.Size([420, 7])
./grab_both_cubes_v1\episode_10.hdf5 successfully saved in ./grab_both_cubes_v2/episode_2.hdf5
saving ./grab_both_cubes_v1\episode_11.hdf5
(489, 6)
torch.Size([489, 7])
./grab_both_cubes_v1\episode_11.hdf5 successfully saved in ./grab_both_cubes_v2/episode_3.hdf5
saving ./grab_both_cubes_v1\episode_12.hdf5
(393, 6)
torch.Size([393, 7])
./grab_both_cubes_v1\episode_12.hdf5 successfully saved in ./grab_both_cubes_v2/episode_4.hdf5
saving ./grab_both_cubes_v1\episode_13.hdf5
(404, 6)
torch.Size([404, 7])
./grab_both_cubes_v1\episode_13.hdf5 successfully saved in ./grab_both_cubes_v2/episo

In [84]:
with h5py.File(save_directory + "/episode_0.hdf5") as f:
    print_hdf5_structure_pure(f)
    print(f['/qpos'][0])
    print(f['action'][0])

Group: /
  Dataset: /action
    Shape: (346, 8)
  Group: /cam_data
    Dataset: /cam_data/human_camera
      Shape: (346, 240, 426, 3)
    Dataset: /cam_data/robot_camera
      Shape: (346, 240, 426, 3)
  Dataset: /end_position
    Shape: (346, 6)
  Dataset: /gripper_state
    Shape: (346,)
  Dataset: /qpos
    Shape: (346, 7)
  Dataset: /qvel
    Shape: (346, 7)
  Dataset: /step
    Shape: (346,)
  Dataset: /timestamp
    Shape: (346,)
  Dataset: /transformed_hand_coords
    Shape: (346, 24, 3)
  Dataset: /transformed_hand_frames
    Shape: (346, 4, 3)
[ 1.5224858e+02  3.0298889e+02  1.4000000e-05 -1.7999991e+02
  0.0000000e+00  8.9999985e+01  1.0000000e+00]
[152.22865 303.19028   0.      180.        0.       90.        1.
   1.     ]


In [68]:
# print some examples 

# with h5py.File(r'./grab_cube2_v2/episode_test.hdf5', 'r') as f:
#     for i in [0,99,199,299]:
#     # for i in [0,99,170]
#         for obj_name in obj_names:
#             if obj_name == "/cam_data/human_camera" :
#                 img = f[obj_name][i]
                
#                 # 使用 matplotlib 显示图像
#                 plt.imshow(img)
#                 plt.title(f"Human Camera - Image {i}")
#                 plt.axis('off')  # 不显示坐标轴
#                 plt.show()
                    
                
                
#             elif obj_name == "/cam_data/robot_camera" :
#                 img = f[obj_name][i]
                
#                 # 使用 matplotlib 显示图像
#                 plt.imshow(img)
#                 plt.title(f"Robot Camera - Image {i}")
#                 plt.axis('off')  # 不显示坐标轴
#                 plt.show()
            
#             else:
#                 print(f"{i}th {obj_name}: {f[obj_name][i]}")
#         print("\n")
    